In [10]:
#verify we have the needed libraries installed using uv
!uv pip install boto3 python-dotenv

Using Python 3.12.1 environment at: /workspaces/TechCatalyst_DE_2025v2/dev4
Audited 2 packages in 12ms


In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
import boto3 
import os

s3_client = boto3.client('s3')

In [13]:
# list bucket names
buckets = s3_client.list_buckets()
for bucket in buckets['Buckets']:
    if 'techcatalyst' in bucket['Name']:
        print(bucket['Name'])

capstone-techcatalyst-conformed
capstone-techcatalyst-raw
capstone-techcatalyst-transformed
techcatalyst-public
techcatalyst-raw
techcatalyst-transformed


In [14]:
# list objects in a specific bucket "techcatalyst-raw" 
bucket_name = 'techcatalyst-raw'
objects = s3_client.list_objects_v2(Bucket= bucket_name)
for obj in objects.get('Contents', []):
    print(obj['Key'])

BLAKE/test.csv
BLAKE/test4.csv
BLAKE/test4.parquet/f02d2bad59de48efa69b0618bd89b99d.snappy.parquet
BLAKE/test_export.parquet
BLAKE/test_snowflake.csv
BLAKE/test_snowflake2.csv
BLAKE/test_snowflake3.csv
BLAKE/upload_file_method_GOOG.csv
BLAKE/upload_file_method_GOOG.parquet/25991cddfcc94a4fb93f5864d77a1fc7.snappy.parquet
BLAKE/upload_fileobj_method.txt
BLAKE_wr/9288e6c1eed4476d98eade7875cbf9c0.snappy.parquet
BLAKE_wr/uploads/wr_newfile.csv
Ben/Million_Songs/
Ben/bingchilling.txt
Ben/gooooog.csv
Ben/parquetGoogleStock/a0eeb97dbd854a5c9cc6e89512faee73.snappy.parquet
Ben/test.csv
Ben/test2.csv
Ben/uploads/Google_upload_test
EMMA/emna_goog.csv
EMMA/emna_goog.txt
EMMA/test_export.parquet
MELISSA/test.csv
MELISSA/uploads/GOOG.csv
MELISSA/uploads/fileobj.txt
MELISSA/uploads/new_file.csv
MillionSongSubset/
MillionSongSubset/log-data/2018-11-01-events.json
MillionSongSubset/log-data/2018-11-02-events.json
MillionSongSubset/log-data/2018-11-03-events.json
MillionSongSubset/log-data/2018-11-04-eve

In [15]:
# list objects that are CSV in a specific bucket "techcatalyst-raw" 
for obj in objects.get('Contents', []):
    if obj['Key'].endswith('.csv'):
        print(obj['Key'])

BLAKE/test.csv
BLAKE/test4.csv
BLAKE/test_snowflake.csv
BLAKE/test_snowflake2.csv
BLAKE/test_snowflake3.csv
BLAKE/upload_file_method_GOOG.csv
BLAKE_wr/uploads/wr_newfile.csv
Ben/gooooog.csv
Ben/test.csv
Ben/test2.csv
EMMA/emna_goog.csv
MELISSA/test.csv
MELISSA/uploads/GOOG.csv
MELISSA/uploads/new_file.csv
aamnah/uploads/GOOG_NEW.csv
accidents/accidents_2017_to_2023_english.csv
alexia/uploads/new_stocks.csv
camrenn.rivera/google_stock_downloaded.csv
fabiola/uploads/wr_GOOG.csv
jaden/test.csv
jaden/uploads/new_file.csv
michael/uploads/new_file_stocks.csv
miraj/test.csv
shaswat/test.csv
shaswat/uploads/test.csv
stage/yellow_tripdata.csv
stocks/GOOG.csv
suchitha/uploads/sales.csv
tatwan/GOOG.csv
tatwan/GOOG_NEW.csv
tyler/last_test.csv
tyler/test.csv
tyler/test2.csv
tyler/test3.csv
tyler/test99.csv
tyler/testidk.csv
tyler/tyler_file.csv
tyler/uploads/goog_stock_file.csv


In [16]:

s3_client.download_file(Bucket='techcatalyst-raw',  # from which bucket
                        Key='stocks/GOOG.csv', # what is the object name, this include the directory/key.csv
                        Filename='GOOG.csv') # Filename is what you want to call it once it is downloaded

In [17]:
import io
io_temp = io.BytesIO()
temp = s3_client.download_fileobj(Bucket='techcatalyst-raw', 
                           			Key='stocks/GOOG.csv',
                            		Fileobj= io_temp) # pass th io.BytesIO object

In [18]:
# show buffer content
print(io_temp.getvalue()[:100])

b'Date,Open,High,Low,Close,Volume\r\n1/2/2025 16:00:00,191.49,193.2,188.71,190.63,17545162\r\n1/3/2025 16:'


In [19]:
io_temp.seek(0)

next(io_temp)

b'Date,Open,High,Low,Close,Volume\r\n'

In [20]:
# writing the content of the BytesIO object to a file 
with open('google_stock_downloaded.csv', 'wb') as f:
    f.write(io_temp.getvalue())

In [21]:
# uploading a local file using upload_file
s3_client.upload_file(Filename='google_stock_downloaded.csv', # local file name
                      Bucket='techcatalyst-raw', # the bucket target
                      Key='camrenn.rivera/google_stock_downloaded.csv') # destination name, make sure it include YOURNAME/ANY_FILE_NAME.csv

In [22]:
in_memory_file = io.BytesIO(b"Io_Upload_Test_Cam")
s3_client.upload_fileobj(Fileobj= in_memory_file,
                           Bucket='techcatalyst-raw', 
                           Key='camrenn.rivera/Io_test_file.txt') # destination name, make sure it include YOURNAME/ANY_FILE_NAME.txt

In [23]:
bucket_name = 'techcatalyst-raw'
objects = s3_client.list_objects_v2(Bucket= bucket_name, Prefix = 'camrenn.rivera')
for obj in objects.get('Contents', []):
    print(obj['Key'])


camrenn.rivera/02a8203f7ed142a38f8c417b15188abc.snappy.parquet
camrenn.rivera/30166a8ceeb64007a46abfbcdea8bd11.snappy.parquet
camrenn.rivera/Io_test_file.txt
camrenn.rivera/google_stock_downloaded.csv


In [24]:
#install awswrangler  
!uv pip install awswrangler

Using Python 3.12.1 environment at: /workspaces/TechCatalyst_DE_2025v2/dev4
Audited 1 package in 14ms


In [25]:
#Load credentials 
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
import awswrangler as wr
# test if you can read from a private bucket
df = wr.s3.read_csv('s3://techcatalyst-raw/stocks/GOOG.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,1/2/2025 16:00:00,191.49,193.20,188.71,190.63,17545162
1,1/3/2025 16:00:00,192.73,194.50,191.35,193.13,12874957
2,1/6/2025 16:00:00,195.15,199.56,195.06,197.96,19483323
3,1/7/2025 16:00:00,198.27,202.14,195.94,196.71,16966760
4,1/8/2025 16:00:00,193.95,197.64,193.75,195.39,14335341


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    140 non-null    object 
 1   Open    140 non-null    float64
 2   High    140 non-null    float64
 3   Low     140 non-null    float64
 4   Close   140 non-null    float64
 5   Volume  140 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 6.7+ KB


In [28]:
databases = wr.catalog.databases()
print(databases)

               Database       Description
0             aamnah_db                  
1           aamnah_taxi                  
2             alexia_db                  
3           alexia_logs                  
4           alexia_song                  
5           alexia_taxi                  
6      awswrangler_test                  
7                ben_db                  
8              ben_song                  
9              ben_taxi                  
10           blake_taxi                  
11          blake_wr_db                  
12    camrenn.rivera_db                  
13              db_name                  
14              default  default database
15              emma_db                  
16            emma_taxi                  
17           fabiola_db                  
18         fabiola_taxi                  
19           jaden_taxi                  
20        jadenastle_db                  
21           melissa_db                  
22         melissa_logs           

In [30]:
name = 'camrenn.rivera'
database_name = f"{name}_db"
#wr.catalog.create_database(database_name)

In [31]:
#inspect again to make sure mine was created
databases = wr.catalog.databases()
print(databases)

               Database       Description
0             aamnah_db                  
1           aamnah_taxi                  
2             alexia_db                  
3           alexia_logs                  
4           alexia_song                  
5           alexia_taxi                  
6      awswrangler_test                  
7                ben_db                  
8              ben_song                  
9              ben_taxi                  
10           blake_taxi                  
11          blake_wr_db                  
12    camrenn.rivera_db                  
13              db_name                  
14              default  default database
15              emma_db                  
16            emma_taxi                  
17           fabiola_db                  
18         fabiola_taxi                  
19           jaden_taxi                  
20        jadenastle_db                  
21           melissa_db                  
22         melissa_logs           

In [32]:
wr.catalog.table(database= 'camrenn.rivera_db', table = 'camrenn_stock')

,Column Name,Type,Partition,Comment
0,date,string,False,
1,open,double,False,
2,high,double,False,
3,low,double,False,
4,close,double,False,
5,volume,bigint,False,


In [33]:
wr.s3.to_parquet(
    df=df, # the DataFrame you just created 
    path=f"s3://techcatalyst-raw/{name}/", # write to the techcatalyst-raw bucket under your folder name (or it would create a new folder if it does not exist)
    dataset=True, 
    database= 'camrenn.rivera_db',#YOUR CODE, # the name of the database you just created in AWS Glue 
    table= 'Camrenn.STOCK', # pick a table name for example YOURNAME_STOCK
    mode='overwrite'
    )

{'paths': ['s3://techcatalyst-raw/camrenn.rivera/4f38082e006e4e5b8df9e8737a975bc4.snappy.parquet'],
 'partitions_values': {}}

In [34]:
wr.catalog.tables(database='camrenn.rivera_db')

,Database,Table,Description,TableType,Columns,Partitions
0,camrenn.rivera_db,camrenn_stock,,EXTERNAL_TABLE,"date, open, high, low, close, volume",


In [35]:
wr.catalog.tables(name_contains="stock")

,Database,Table,Description,TableType,Columns,Partitions
0,aamnah_db,aamnah_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
1,alexia_db,alexia_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
2,ben_db,ben_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
3,blake_wr_db,blake_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
4,camrenn.rivera_db,camrenn_stock,,EXTERNAL_TABLE,"date, open, high, low, close, volume",
5,emma_db,emma_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
6,fabiola_db,fabiola_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
7,jadenastle_db,jaden_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
8,melissa_db,melissa_stocks,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
9,michael_db,michael_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",


In [36]:
df = wr.s3.read_parquet_table(database='camrenn.rivera_db', 
                              table='camrenn_stock')

# Display the DataFrame's first few rows
df.head()

,date,open,high,low,close,volume
0,1/2/2025 16:00:00,191.49,193.20,188.71,190.63,17545162
1,1/3/2025 16:00:00,192.73,194.50,191.35,193.13,12874957
2,1/6/2025 16:00:00,195.15,199.56,195.06,197.96,19483323
3,1/7/2025 16:00:00,198.27,202.14,195.94,196.71,16966760
4,1/8/2025 16:00:00,193.95,197.64,193.75,195.39,14335341


In [37]:
wr.catalog.get_table_types(database='camrenn.rivera_db', 
                           table='camrenn_stock')

{'date': 'string',
 'open': 'double',
 'high': 'double',
 'low': 'double',
 'close': 'double',
 'volume': 'bigint'}

In [38]:
table_details = wr.catalog.get_tables(database='camrenn.rivera_db')

next(table_details)

{'Name': 'camrenn_stock',
 'DatabaseName': 'camrenn.rivera_db',
 'CreateTime': datetime.datetime(2025, 8, 5, 14, 14, 33, tzinfo=tzlocal()),
 'UpdateTime': datetime.datetime(2025, 8, 5, 14, 14, 33, tzinfo=tzlocal()),
 'Retention': 0,
 'StorageDescriptor': {'Columns': [{'Name': 'date', 'Type': 'string'},
   {'Name': 'open', 'Type': 'double'},
   {'Name': 'high', 'Type': 'double'},
   {'Name': 'low', 'Type': 'double'},
   {'Name': 'close', 'Type': 'double'},
   {'Name': 'volume', 'Type': 'bigint'}],
  'Location': 's3://techcatalyst-raw/camrenn.rivera/',
  'InputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat',
  'OutputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat',
  'Compressed': True,
  'NumberOfBuckets': -1,
  'SerdeInfo': {'SerializationLibrary': 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe',
   'Parameters': {'serialization.format': '1'}},
  'BucketColumns': [],
  'SortColumns': [],
  'Parameters': {'CrawlerSchemaDes

In [41]:
# Example adding additional metadata information 

desc = "This is my stock table."
param = {"source": "Google", "class": "stock"}
comments = {
    "Date": "Trading Date",
    "Open": "Opening Price",
    "Close": "Closing Price"
}

wr.s3.to_parquet(
    df=df,
    path='s3://techcatalyst-raw/camrenn.rivera',#BUCKET/FOLDER, # CHANGE THIS TO USE YOUR NAME for example s3://TECHCATALYST/TATWAN
    dataset=True,
    database='camrenn.rivera_db',#YOURD_DATABASENAME'',
    table='camrenn_stock',
    mode='overwrite',
    glue_table_settings=wr.typing.GlueTableSettings(description=desc,  # here we are passing some metadata
                                                    parameters=param, 
                                                    columns_comments=comments),
    )

{'paths': ['s3://techcatalyst-raw/camrenn.rivera/103580d26d4e41f28117378aef87be24.snappy.parquet'],
 'partitions_values': {}}

In [42]:
wr.catalog.table(database='camrenn.rivera_db', table='camrenn_stock')

,Column Name,Type,Partition,Comment
0,date,string,False,Trading Date
1,open,double,False,Opening Price
2,high,double,False,
3,low,double,False,
4,close,double,False,Closing Price
5,volume,bigint,False,


In [45]:
wr.s3.list_objects('s3://techcatalyst-raw/stage/')

['s3://techcatalyst-raw/stage/yellow_tripdata.csv',
 's3://techcatalyst-raw/stage/yellow_tripdata.json',
 's3://techcatalyst-raw/stage/yellow_tripdata.parquet']

In [46]:
wr.s3.download(path='s3://techcatalyst-raw/stocks/GOOG.csv', 
               local_file='./new_file.csv')

In [47]:
your_name = 'camrenn.rivera'
file_name = 'test_wr_upload'#ENTER YOUR FILE NAME YOU WANT TO UPLOAD
wr.s3.upload(local_file='new_file.csv',path= f's3://techcatalyst-raw/{your_name}/uploads/{file_name}')

In [49]:
wr.s3.list_objects(f's3://techcatalyst-raw/{your_name}/uploads/')

['s3://techcatalyst-raw/camrenn.rivera/uploads/test_wr_upload']

In [51]:
# A name for the new Glue Database it should be YOURNAME_TAXI
db_name = 'camrenn.rivera_TAXI' # YOUR CODE

# A name for the table it should be YOURNAME_TRIPDATA
table_name =  'camrenn.rivera_TRIPDATA'#YOUR CODE

# the path_direcoty to should point to the bucket (main directory)
s3_path_directory = 's3://techcatalyst-raw/taxi_data/'

# The path_file should be the full path to the actual file
s3_path_file = 's3://techcatalyst-raw/yellow_tripdata_2024-01.parquet'


In [52]:
# uncomment below if you ran into issues to clean things up and rerun the cell
wr.catalog.delete_table_if_exists(database=db_name, table=table_name) 

# Create the new Glue database first based on the db_name you created
# YOUR CODE
wr.catalog.create_database(db_name)
# This function can extract the schema from our file and returns a tuple: (schema, partitions). We only need the schema. 
columns_types, partitions_types = wr.s3.read_parquet_metadata(path=s3_path_file)
print("Successfully read schema from Parquet file.")

Successfully read schema from Parquet file.


In [53]:

wr.catalog.create_parquet_table(
    database=db_name, # pass the database name
    table=table_name, # pass the table name
    path=s3_path_directory, # use the directoy here 
    columns_types=columns_types,  # Pass the schema here
    partitions_types=partitions_types
)
print(f"Table '{table_name}' created successfully in database '{db_name}'.")

Table 'camrenn.rivera_TRIPDATA' created successfully in database 'camrenn.rivera_TAXI'.


In [61]:
s3_client.upload_file(Filename='test.csv', # local file name
                      Bucket='techcatalyst-raw', # the bucket target
                      Key='camrenn.rivera/test_v3.csv') # destination name, make sure it include YOURNAME/ANY_FILE_NAME.csv
